<a href="https://colab.research.google.com/github/caramos84/2674492/blob/main/CalculadoraPesoIdeal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Calculadora de peso ideal
# Verificación de si una persona está o no en un buen peso
# Carlos Ramos + gpt-o1
# 10/02/2025

import numpy as np

# Función de activación: tangente hiperbólica
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

# Softmax para la capa de salida
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / exp_x.sum(axis=1, keepdims=True)

# Datos de entrenamiento ampliados
X = np.array([
    [1.45, 40], [1.50, 45], [1.55, 50], [1.60, 55],  # Bajo peso
    [1.65, 60], [1.70, 68], [1.75, 75], [1.80, 85],  # Peso ideal
    [1.85, 95], [1.90, 105], [1.95, 115]            # Sobrepeso
])

# Etiquetas en One-Hot Encoding
Y = np.array([
    [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0],  # Bajo peso
    [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0],  # Peso ideal
    [0, 0, 1], [0, 0, 1], [0, 0, 1]             # Sobrepeso
])

# Normalización (Estandarización Z-score)
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

# Inicialización de pesos (Xavier Initialization)
W1 = np.random.randn(2, 6) * np.sqrt(2 / 2)  # Aumentamos a 6 neuronas en capa oculta
W2 = np.random.randn(6, 3) * np.sqrt(2 / 6)

# Hiperparámetros
lr = 0.3
epochs = 15000

# Entrenamiento
for epoch in range(epochs):
    # Forward Pass
    hidden_input = np.dot(X, W1)
    hidden_output = tanh(hidden_input)

    final_input = np.dot(hidden_output, W2)
    final_output = softmax(final_input)

    # Cálculo de error
    error = Y - final_output

    # Backpropagation
    d_output = error
    d_hidden = np.dot(d_output, W2.T) * tanh_derivative(hidden_output)

    # Actualización de pesos
    W2 += np.dot(hidden_output.T, d_output) * lr
    W1 += np.dot(X.T, d_hidden) * lr

    # Muestra de error cada 1000 iteraciones
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Error: {np.mean(np.abs(error)):.5f}")

# Predicción final con nuevos datos
test_data = np.array([[1.55, 52], [1.73, 72], [1.87, 100]])

# Aplicar la misma normalización a los datos de prueba
test_data = (test_data - X_mean) / X_std

hidden_layer = tanh(np.dot(test_data, W1))
output_layer = softmax(np.dot(hidden_layer, W2))

# Definiendo categorías
categories = ["Bajo peso", "Peso ideal", "Sobrepeso"]

for i in range(len(test_data)):
    prediction = np.argmax(output_layer[i])
    print(f"Altura: {test_data[i][0] * X_std[0] + X_mean[0]:.2f} m, "
          f"Peso: {test_data[i][1] * X_std[1] + X_mean[1]:.2f} kg → "
          f"Categoría: {categories[prediction]}")

Epoch 0, Error: 0.41960
Epoch 1000, Error: 0.00060
Epoch 2000, Error: 0.00028
Epoch 3000, Error: 0.00018
Epoch 4000, Error: 0.00013
Epoch 5000, Error: 0.00010
Epoch 6000, Error: 0.00008
Epoch 7000, Error: 0.00007
Epoch 8000, Error: 0.00006
Epoch 9000, Error: 0.00005
Epoch 10000, Error: 0.00005
Epoch 11000, Error: 0.00004
Epoch 12000, Error: 0.00004
Epoch 13000, Error: 0.00003
Epoch 14000, Error: 0.00003
Altura: 1.55 m, Peso: 52.00 kg → Categoría: Bajo peso
Altura: 1.73 m, Peso: 72.00 kg → Categoría: Peso ideal
Altura: 1.87 m, Peso: 100.00 kg → Categoría: Sobrepeso
